In [1]:
import sys
sys.path.append('../../')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import redis
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (6, 6)
import tensorflow as tf
import numpy as np
from distribution.watcher import NoSynchronization
from machine_learning.models import DoubleProduction, DoubleConcatProduction, DoubleExtraProduction, DoubleExtraConcatProduction
tf.random.set_seed(42)

In [2]:
import math 

def std_deviation(lst):
    mean = avg(lst)
    sum = 0
    for i in lst:
        sum += (float(i.decode("utf-8")) - mean)**2

    return math.sqrt(sum / len(lst))
    

In [3]:
model_list = [DoubleProduction, DoubleConcatProduction, DoubleExtraProduction, DoubleExtraConcatProduction]
r = redis.Redis(host='localhost', port=6379, db=0)
def sum(lst):
    sum = 0
    for i in lst:
        sum += float(i.decode("utf-8"))
    return sum

def avg(lst):
    sum = 0
    for i in lst:
        if type(i) != float:
            sum += float(i.decode("utf-8"))
        else:
            sum += i
    return sum / len(lst)

import math 

def std_deviation(lst):
    mean = avg(lst)
    sum = 0
    for i in lst:
        if type(i) != float:
            sum += (float(i.decode("utf-8")) - mean)**2
        else:
            sum += (i - mean)**2
    return math.sqrt(sum / len(lst))

def func_f1_score(lst1, lst2):
    res = []
    for prc, rc in zip(lst1, lst2):
        if type(prc) != float or type(rc) != float:
            prc = float(prc.decode("utf-8"))
            rc = float(rc.decode("utf-8"))

        res.append(2 * (prc * rc) / (prc + rc))
    return res

In [4]:
loss = {}
recall = {}
precision = {}
f1_score = {}
auc = {}
throughput = {}
total_time = {}
avg_pass_time = {}
read_time = {}
write_time = {}

loss_dev = {}
recall_dev = {}
precision_dev = {}
f1_score_dev = {}
auc_dev = {}
throughput_dev = {}
total_time_dev = {}
avg_pass_time_dev = {}
read_time_dev = {}
write_time_dev = {}


threshold_limit = [1, 2, 4, 8, 16, 32, 64, 128,256, 512, 1024]
num_workers = [2, 4, 8]
x_scale = ["2", "4", "8"]
labels = {
    'double_production': 'double GRU',
    'double_extra_production': 'double GRU + extra layer',
    'double_concat_production': 'double GRU + skip connection',
    'double_extra_concat_production': 'double GRU + extra layer + skip connection'
}

for model_name, label in labels.items():
    loss[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Loss_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    recall[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Recall_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    precision[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Precision_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    auc[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Auc_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]

    f1_score[model_name] = [avg(func_f1_score(
            r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Precision_thres_{1}_pool_{pool}', 0, -1),
            r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Recall_thres_{1}_pool_{pool}', 0, -1)
    )) for pool in num_workers]

    throughput[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Throughput_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    total_time[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: TotalTime_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    avg_pass_time[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: AvgPass_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    read_time[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: read_time_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    write_time[model_name] = [avg(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: write_time_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]

    loss_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Loss_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    recall_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Recall_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    precision_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Precision_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    auc_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Auc_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]

    throughput_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Throughput_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    total_time_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: TotalTime_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    avg_pass_time_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: AvgPass_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    read_time_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: read_time_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    write_time_dev[model_name] = [std_deviation(r.lrange(f'v1_{model_name}_{NoSynchronization.name}: write_time_thres_{1}_pool_{pool}', 0, -1)) for pool in num_workers]
    f1_score_dev[model_name] = [std_deviation(func_f1_score(
            r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Precision_thres_{1}_pool_{pool}', 0, -1),
            r.lrange(f'v1_{model_name}_{NoSynchronization.name}: Recall_thres_{1}_pool_{pool}', 0, -1)
    )) for pool in num_workers]


In [5]:
#dataframe = {
#    'num_workers': [2, 4, 8, 2, 4, 8, 2, 4, 8, 2, 4, 8],
#    'model': ['double_production','double_extra_production','double_concat_production','double_extra_concat_production',
#              'double_production','double_extra_production','double_concat_production','double_extra_concat_production',
#              'double_production','double_extra_production','double_concat_production','double_extra_concat_production']
#    'loss': 
#}

In [6]:
for model_name, label in labels.items():
    print("\t\t2\t\t4\t\t8")
    print(model_name, loss[model_name])
    print(model_name, loss_dev[model_name])


		2		4		8
double_production [0.021021654829382896, 0.021036045160144567, 0.021091619739308953]
double_production [0.0003932453691959381, 0.0007965692335388816, 0.0010511404192249927]
		2		4		8
double_extra_production [0.01549295149743557, 0.015392089728266, 0.015331293572671711]
double_extra_production [6.767362356185913e-05, 0.0013344185620630502, 0.0015134783599226518]
		2		4		8
double_concat_production [0.01576287392526865, 0.01576707442291081, 0.015770515892654657]
double_concat_production [7.599685341119766e-05, 0.0010243991128394588, 0.0015089957384469823]
		2		4		8
double_extra_concat_production [0.017265682108700275, 0.017273544799536467, 0.017285619629547]
double_extra_concat_production [6.292108446359634e-05, 0.0012618676417713007, 0.0018168877351508912]


In [7]:
for model_name, label in labels.items():
    print("\t\t2\t\t4\t\t8")
    print(model_name, precision[model_name])
    print(model_name, precision_dev[model_name])

		2		4		8
double_production [0.3223593235015869, 0.3230437785387039, 0.3249813988804817]
double_production [0.03228297829627991, 0.0325985867756776, 0.05059214107341371]
		2		4		8
double_extra_production [0.5343746542930603, 0.5290040746331215, 0.5493813455104828]
double_extra_production [0.02246987819671631, 0.04916136620826865, 0.08468398978408738]
		2		4		8
double_concat_production [0.5111136138439178, 0.5100044012069702, 0.5106570720672607]
double_concat_production [0.003421276807785034, 0.02181639956151032, 0.07023796438115588]
		2		4		8
double_extra_concat_production [0.3840796500444412, 0.3821539655327797, 0.3896446619182825]
double_extra_concat_production [0.020798400044441223, 0.041699638875145356, 0.12245895594986815]


In [8]:
for model_name, label in labels.items():
    print("\t\t2\t\t4\t\t8")
    print(model_name, recall[model_name])
    print(model_name, recall_dev[model_name])


		2		4		8
double_production [0.37515710294246674, 0.3780280277132988, 0.3836948163807392]
double_production [0.0450412780046463, 0.04806076339530305, 0.0591181526778477]
		2		4		8
double_extra_production [0.17046428099274635, 0.1768667921423912, 0.1837964216247201]
double_extra_production [0.04594690725207329, 0.05131283542991707, 0.047722247655536475]
		2		4		8
double_concat_production [0.17469432950019836, 0.17475923895835876, 0.17532279714941978]
double_concat_production [0.003844901919364929, 0.014704815777729805, 0.02550345454742709]
		2		4		8
double_extra_concat_production [0.08198758214712143, 0.082126934081316, 0.0814207918010652]
double_extra_concat_production [0.001871749758720398, 0.012490232538506056, 0.02301935308582139]


In [9]:
for model_name, label in labels.items():
    print("\t\t2\t\t4\t\t8")
    print(model_name, f1_score[model_name])
    print(model_name, f1_score_dev[model_name])

		2		4		8
double_production [0.3467253705957508, 0.34803254228501007, 0.3510164148871288]
double_production [0.0379217558377678, 0.03815958894216728, 0.05113277973585259]
		2		4		8
double_extra_production [0.2559994378376427, 0.26285553606113865, 0.27217133599420196]
double_extra_production [0.05568887874994391, 0.06506823475822889, 0.05902806161253025]
		2		4		8
double_concat_production [0.2603489382545341, 0.26019872403220956, 0.26030193593885315]
double_concat_production [0.003827190675288056, 0.01877588938473243, 0.03438015430505978]
		2		4		8
double_extra_concat_production [0.13501341634529115, 0.1350553529909555, 0.13375652928020004]
double_extra_concat_production [0.0012503188275287125, 0.019098375713953678, 0.036673987912012346]


In [11]:
for model_name, label in labels.items():
    print("\t\t2\t\t4\t\t8")
    print(model_name, auc[model_name])
    print(model_name, auc_dev[model_name])


		2		4		8
double_production [0.9479008913040161, 0.9480039775371552, 0.9483500272035599]
double_production [0.010629773139953613, 0.01158102574132103, 0.014782952792487354]
		2		4		8
double_extra_production [0.853668212890625, 0.8527589589357376, 0.8506893515586853]
double_extra_production [0.009079933166503906, 0.009280851727143906, 0.021856997866815917]
		2		4		8
double_concat_production [0.8572530150413513, 0.857427716255188, 0.8572641611099243]
double_concat_production [0.008143723011016846, 0.008906820970045495, 0.011693666082939331]
		2		4		8
double_extra_concat_production [0.8440549969673157, 0.8436242789030075, 0.8433194234967232]
double_extra_concat_production [0.006097137928009033, 0.006814688607689127, 0.009482092053713326]
